In [1]:
# Script to Timeseries ROS => Features

In [2]:
import pandas as pd
import numpy as np
import os
import regex as re

In [3]:
participant = 18
data_path = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/ROS/ROS_Sessions/p'+str(participant) +'/TS'
out = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/ROS/ROS_Sessions/p'+str(participant) +'/Final/'

In [4]:
for subdir, dirs, files in os.walk(data_path):
    for file in files:
        if 'ros' in file and 'csv' in file:
            print(file)
                        
            sess = pd.read_csv(data_path+'/'+file)
            sess = sess.sort_values(['timestamp'], ascending=[True])
            
            # 1. Precondition Calculations
            print('1--Precondition Calculations')
            sess['mistake_made'] = 0
            sess['game_incorrect'] = 0
            sess['game_correct'] = 0
            sess['game_start'] = 0

            for i,r in sess.iterrows():
                ros_robot = r['ROBOT STATE']
                ros_participant = r['PARTICIPANT STATE']

                # new continuous instance in ROBOT STATE 
                if (isinstance(ros_robot, str)) and ((i==0) or (sess.at[i-1, 'ROBOT STATE'] != ros_robot)): 
                    # new game_start
                    if ('game' in ros_robot) and ('act' in ros_robot) and  ('inst' in ros_robot):
                        sess.at[i, 'game_start'] = 1

                # new continuous instance in PARTICIPANT STATE
                if (isinstance(ros_participant, str)) and ((i==0) or (sess.at[i-1, 'PARTICIPANT STATE'] != ros_participant)):
                    # new game correct 
                    if ('Game Completed' in ros_participant):
                        sess.at[i, 'game_correct'] = 1

                    # new game incorrect
                    if ('Moving on' in ros_participant):
                        sess.at[i, 'game_incorrect'] = 1

                    # mistake made
                    if ('Made a mistake' in ros_participant):
                        sess.at[i, 'mistake_made'] = 1
                        
            # 2. Interaction Calculations
            print('2--Interaction Calculations')
            sess['ts_robot_talked'] = 0
            sess['ts_game_start'] = 0
            sess['ts_attempt'] = 0
            sess['games_session'] = 0
            sess['mistakes_session'] = 0
            sess['mistakes_game'] = 0

            # time since robot talked
            last_robot = 0.0

            # time since game started (elapsed time on current attempt)
            game_started = False
            last_game_start = 0.0
            last_game_finished = 0.0

            # games, mistakes 
            games_session = 0
            mistakes_session = 0
            mistakes_game = 0

            # time since last recorded attempt
            last_record = 0.0

            # iterate through all rows, tracking new features :) 
            for i,r in sess.iterrows():
                # time since robot talked (initialized to 0)
                if isinstance(r['ROBOT STATE'], str):
                    last_robot = r['timestamp']
                else:
                    sess.loc[i, 'ts_robot_talked'] = r['timestamp'] - last_robot

                # time since game started (initialized to 0)
                if (r['game_start'] == 1):
                    last_game_start = r['timestamp']
                    game_started = True

                # we have a recorded attempt! 
                if (r['game_start'] == 1) or (r['mistake_made'] == 1):
                    last_record = r['timestamp']

                # need to check whether game hasn't finished yet (time between games == 0)
                if (game_started) and (last_game_finished <= last_game_start):
                    sess.loc[i, 'ts_game_start'] = r['timestamp'] - last_game_start
                    sess.loc[i, 'ts_attempt'] = r['timestamp'] - last_record

                if (r['game_correct'] == 1) or (r['game_incorrect'] == 1):
                    last_game_finished = r['timestamp']

                # games total calculations
                if (r['game_start'] == 1):
                    games_session += 1
                sess.loc[i, 'games_session'] = games_session

                # mistakes calculations
                if (r['mistake_made'] == 1):
                    mistakes_session += 1
                    mistakes_game += 1
                sess.loc[i, 'mistakes_session'] = mistakes_session
                sess.loc[i, 'mistakes_game'] = mistakes_game

                if (r['game_correct'] == 1) or (r['game_incorrect'] == 1):        
                    mistakes_game = 0

                    
            # 3. Game Calculations  
            print('3--Game Calculations')

            act_pattern = 'GAME ID: \d+'
            diff_pattern = 'DIFFICULTY: \d+'

            sess['activity'] = np.nan
            sess['difficulty'] = np.nan
            sess['skill'] = np.nan
            sess['aptitude'] = 0.5

            sess['diff_1'] = 0
            sess['diff_2'] = 0
            sess['diff_3'] = 0
            sess['diff_4'] = 0
            sess['diff_5'] = 0

            sess['skill_EM'] = 0
            sess['skill_NC'] = 0
            sess['skill_EM'] = 0
            sess['in_game'] = np.nan

            act_to_skill = {
                1: 'NC',
                2: 'NC',
                3: 'NC',
                4: 'NC',
                5: 'NC',
                6: 'NC',
                7: 'OS',
                8: 'OS',
                9: 'OS',
                10: 'OS',
                16: 'EM',
            }

            for i,r in sess.iterrows():
                if r['ts_robot_talked']==0 or r['ts_game_start']==0:
                    sess.loc[i, 'in_game'] = 0
                else:
                    sess.loc[i, 'in_game'] = 1

                s = r['GAME STATE']
                if pd.isna(s):
                    continue

                act = re.findall(act_pattern, s)[0]
                diff = re.findall(diff_pattern, s)[0]
                
                act = int(act.replace('GAME ID: ', ''))
                diff = int(diff.replace('DIFFICULTY: ', ''))
                
                sess.loc[i, 'activity'] = act
                sess.loc[i, 'difficulty'] = diff
                
                sess.loc[i, 'skill'] = act_to_skill[act]

                sess.loc[i, 'diff_'+str(diff)] = 1
                sess.loc[i, 'skill_'+str(act_to_skill[act])] = 1

                # aptitude calculation
                d = diff / 5.0;
                m = r['mistakes_game'] / 5.0;
                sess.loc[i, 'aptitude'] = 0.5*((1.0-m)+d)
        
            # Write To Output
            sess.to_csv(out+file, index=False)
            print()

p18_s13_ros.csv
1--Precondition Calculations
2--Interaction Calculations
3--Game Calculations



### Precondition Calculations
- game start
    - new continuous instance of "game2_act1_inst0" in ros_ROBOT_STATE
- game end (moving on)
    - "Movingon" in ros_PARTICIPANT_STATE
- game end (completed successfully)
    - "GameCompleted" in ros_PARTICIPANT_STATE
- mistake made
    - new continuous instance of "Madeamistake in ros_PARTICIPANT_STATE"